# Reading in the experiment to analyze for bit error, codebooks, and bulk rna-seq data

In [2]:
analysis_folder = r"D:\MERFISH\Archived_Experiments\E20190919_R0003\MERlin_Analysis\E20190919_R0003_v2"
codebook_file = r"D:\MERFISH\MERlin_Parameters\codebooks\codebook_alex_130gene.csv"
#codebook_file = r"D:\MERFISH\MERlin_Parameters\codebooks\codebook_heart_tissue.csv"
expanded_codebook_file = r"D:\MERFISH\MERlin_Parameters\codebooks\codebook_alex_130gene_expanded.csv"
#expanded_codebook_file = r"D:\MERFISH\MERlin_Parameters\codebooks\codebook_heart_tissue_expanded.csv"
rnaseq_reference = r'D:\MERFISH\Random_Data\U2OS_abundances_qz.rpkm'

In [3]:
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
import os
import PIL
from PIL import Image


In [4]:
print("Counting decoded barcodes")
barcode_files = glob.glob(os.path.join(analysis_folder, r"Decode\barcodes\barcode_data_*.h5"))
fovs = len(barcode_files)
raw_count = 0
for file in tqdm(glob.glob(os.path.join(analysis_folder, r"Decode\barcodes\barcode_data_*.h5"))):
    barcodes = pd.read_hdf(file)
    raw_count += len(barcodes)
    
print("Counting filtered barcodes")
filter_count = len(pd.read_csv(os.path.join(analysis_folder, r"ExportBarcodes\barcodes.csv")))

print("FOVS:", fovs)
print(raw_count, raw_count/fovs, filter_count, filter_count/fovs, filter_count/raw_count)

Counting decoded barcodes


KeyboardInterrupt: 

In [ ]:
import pandas
from scipy.linalg import norm
from sklearn.neighbors import NearestNeighbors
from collections import Counter

def process_fov(filename, normcodes):
    barcodes = pandas.read_hdf(filename)
    fov = int(filename.split('_')[-1].split('.')[0])

    #Subset for testing
    #barcodes = barcodes.iloc[1000:1050]

    #Get just the intensity columns for convenience
    intensities = barcodes[[f'intensity_{i}' for i in range(16)]]
    intensities = intensities.rename(columns=lambda x: 'intensity_' + str(int(x.split('_')[1])+1)) 

    #Find nearest
    neighbors = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
    neighbors.fit(normcodes)
    distances, indexes = neighbors.kneighbors(intensities, return_distance=True)

    rows = []
    for i, index in enumerate(indexes):
        barcode = codebook.iloc[index[0]]
        ints = intensities.iloc[i]
        test = pandas.concat([barcode, ints], axis=0)
        rows.append(test)

    df = pandas.DataFrame(rows)
    df['fov'] = fov
    
    return df

def error_stats(data):
    c = Counter(data.filter(like='bit').sum(axis=1))
    total = sum(c.values())
    return pandas.Series([total, c[4] / total, c[5] / total, c[3] / total], index=["Barcodes", "Correct", "0->1 error", "1->0 error"])
    

In [ ]:
#L2 normalize codebook
import pandas

codebook = pandas.read_csv(expanded_codebook_file)
codes = codebook.filter(like='bit')
normcodes = codes.apply(lambda row: row / norm(row), axis=1)

files = glob.glob(os.path.join(analysis_folder, r"AdaptiveFilterBarcodes\barcodes\barcode_data_*.h5"))

dfs = [process_fov(filename, normcodes) for filename in tqdm(files)]
df = pandas.concat(dfs)

# Error Rate Data

In [ ]:
#This will give the 0 -> 1 and 1 -> 0 error rates
print(error_stats(df))

In [ ]:
ones2zeros = []
zeros2ones = []
for gene in df['name'].unique():
    x = df[df['name'] == gene]
    z = codebook[codebook['id'] == gene]
    i = z.columns[(z == 1).iloc[0]]
    o2z = pandas.DataFrame((len(x) - x[i].sum()) / len(x)).T
    z2o = pandas.DataFrame(x[x.filter(like='bit').columns.difference(i)].sum() / len(x)).T
    ones2zeros.append(o2z)
    zeros2ones.append(z2o)

The following cell will export the bit errors to an excel sheet; change the name of the excel sheet if wanted in the ExcelWriter function. It will also give error visualizations

In [ ]:
import seaborn as sns
import math
import matplotlib.pyplot as plt

def meltdata(df):
    data = pandas.concat(df, axis=0)
    data = data.rename(columns=lambda x: int(x.strip('bit')))
    data = data.melt(value_vars=data.columns, var_name='Bit', value_name='Error rate')
    return data

d = meltdata(ones2zeros)
d['Error type'] = '1 -> 0'
d2 = meltdata(zeros2ones)
d2['Error type'] = '0 -> 1'
data = pandas.concat([d, d2])

ax = sns.catplot(x="Bit", y="Error rate", row="Error type", data=data, kind="bar", height=2, aspect=3,
    capsize=0.2, color='dodgerblue', order=sorted(data["Bit"].unique()), sharey=False, sharex=False)
ax.savefig(os.path.join(analysis_folder, "biterror.png"), dpi=300)
average = {}
average2 = {}
for index, i in d.iterrows():
    average[(i["Bit"])] = []
for index, i in d.iterrows():
    average.get(i["Bit"]).append(i["Error rate"])
for i in average:
    thelist = average.get(i)
    countj = 0
    runningj = 0
    for j in thelist:
        if math.isnan(j) == False:
            runningj += j
            countj += 1
    if countj > 0:
        average2[i] = float(runningj/countj)
average2 = pd.DataFrame(average2.items())
average2.columns = ["Bit", "Error"]
print(average2)

average01 = {}
for index, i in d2.iterrows():
    average01[(i["Bit"])] = []
for index, i in d2.iterrows():
    average01.get(i["Bit"]).append(i["Error rate"])
for i in average01:
    thelist = average01.get(i)
    countj = 0
    runningj = 0
    for j in thelist:
        if math.isnan(j) == False:
            runningj += j
            countj += 1
    if countj > 0:
        average01[i] = float(runningj/countj)
average01 = pd.DataFrame(average01.items())
average01.columns = ["Bit", "Error"]
print(average01)

with pd.ExcelWriter('3.xlsx') as writer:  
    average2.to_excel(writer, sheet_name='Sheet_name_1')
    average01.to_excel(writer, sheet_name='Sheet_name_2')

# Finding the error rate per FOV

In [ ]:
fovs = df.groupby('fov').apply(error_stats)
fovs['FOV'] = fovs.index
fovs.columns = ['Count', 'Correct', '0 -> 1', '1 -> 0', 'FOV']
fovs = fovs.melt(id_vars='FOV', value_vars=['0 -> 1', '1 -> 0'], var_name = 'Error type', value_name='Error rate')

In [ ]:
def fovplot(d, ax):
    q1 = d['Error rate'].quantile(0.25)
    q3 = d['Error rate'].quantile(0.75)
    outlier = q3 + 1.5 * (q3 - q1)
    outlier
    ax.bar(d['FOV'], d['Error rate'], width=1)
    ax.axhline(outlier, color='red', linestyle=':')
    for i, row in d[d['Error rate'] >= outlier].iterrows():
        ax.annotate(row['FOV'], (row['FOV'], row['Error rate']), ha='center', color='orangered', weight='heavy')
    #ax.set_xlim(left=0, right=d['FOV'].max())
    ax.set_xticks(list(range(0, d['FOV'].max(), 25)) + [d['FOV'].max()])
    ax.set_xlabel("FOV")
    ax.set_ylabel("Error rate")
    ax.set_ylim(bottom=d['Error rate'].min()*0.75)
        
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(7,5))
fovplot(fovs[fovs['Error type'] == '0 -> 1'], ax[0])
fovplot(fovs[fovs['Error type'] == '1 -> 0'], ax[1])
ax[0].set_title("Error type = 0 -> 1")
ax[1].set_title("Error type = 1 -> 0")
plt.tight_layout()
plt.savefig(os.path.join(analysis_folder, "foverror.png"), dpi=300)

# Gene abundance Correlations

Can add gene names in using if you want to see their normalized counts

In [ ]:
import math
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

def plot_correlation(xcounts, ycounts, xlabel, ylabel, outfile, omit=[]):
    set1 = set(xcounts.keys())
    set2 = set(ycounts.keys())
    genes_to_consider = list(set1.intersection(set2))
    x = [xcounts[gene] for gene in genes_to_consider if gene not in omit]
    y = [ycounts[gene] for gene in genes_to_consider if gene not in omit]
    using = []
    #using.append("PRPF8")

    for gene in genes_to_consider: 
        if gene not in omit and gene in using:
            print(gene, (10**ycounts[gene]))
    num = []
    count = 0
    for gene in genes_to_consider:
    #    if gene not in omit:
    #        count += 1
            #print(gene)
    #        if gene == "EGFR":
    #            print(10**y[count])
            if gene not in omit:
                num.append(gene) 
    
    corr, pval = pearsonr(x,y)
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.figure(figsize=(7,5))
    plt.plot(x,p(x),"r--")

    plt.scatter(x, y)
    plt.title("Pearson = %.3f" %corr)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    for i, txt in enumerate(num):
        plt.annotate(txt, (x[i], y[i]), size=8, textcoords="offset points")
    #plt.savefig(os.path.join(analysis_folder, outfile), dpi=300)
    

Loading the merfish counts from our experiments, whichever barcode_counts.npy file that is desired

In [ ]:
import os
import numpy as np
import pandas as pd

def load_merfish_counts(filename, codebook):
    gene_counts = np.load(filename)
    ordered_genes = list(codebook['name'])
    abundances = {}
    for i, gene in enumerate(ordered_genes):
        abundances[gene] = np.log10(gene_counts[i]+1)
    return abundances

#Load MERFISH counts
codebook = pd.read_csv(codebook_file)
#merfish_counts = load_merfish_counts(os.path.join(analysis_folder, r"PlotPerformance\filterplots\FilteredBarcodesMetadata\barcode_counts.npy"), codebook)
merfish_counts = load_merfish_counts(r"D:\MERFISH\E20190920_R0004\MERlin_Analysis\E20190920_R0004_v4\PlotPerformance\filterplots\FilteredBarcodesMetadata\barcode_counts.npy",codebook)
#merfish_counts = load_merfish_counts(r"D:\MERFISH\MERlin_Analysis\E20201123_R0040\PlotPerformance\filterplots\FilteredBarcodesMetadata\barcode_counts.npy",codebook)
merfish_counts = load_merfish_counts(r"D:\MERFISH\MERlin_Analysis\E20201223_R0049_S2\PlotPerformance\filterplots\FilteredBarcodesMetadata\barcode_counts.npy",codebook)
#merfish_counts = load_merfish_counts(r"C:\Users\amonell\Documents\barcode_counts.npy",codebook)

Loading in the Moffitt PNAS rna-seq counts (old version)

In [ ]:
pnas_rna = pd.read_csv(r'D:\MERFISH\Random_Data\SRR987281.fpkm', sep=',')
pnas_rna2 = pd.read_csv(r'D:\MERFISH\Random_Data\SRR987282.fpkm', sep=',')
pnas_rna3 = pd.read_csv(r'D:\MERFISH\Random_Data\SRR987283.fpkm', sep=',')
pnas1 = dict(zip(pnas_rna['gene_id'], pnas_rna['FPKM']))
pnas2 = dict(zip(pnas_rna2['gene_id'], pnas_rna2['FPKM']))
pnas3 = dict(zip(pnas_rna3['gene_id'], pnas_rna3['FPKM']))
setpnas1 = set(pnas1.keys())
setpnas2 = set(pnas2.keys())
setpnas3 = set(pnas3.keys())
genes_to_consider2 = setpnas1.intersection(setpnas2)
genes_to_consider3 = list(genes_to_consider2.intersection(setpnas3))
fin_dict = {}
for gene in genes_to_consider3:
    avg = (pnas1[gene] + pnas2[gene] + pnas3[gene])/3.0
    fin_dict[gene] = np.log10(avg+1)
    #fin_dict[gene] = avg + 1

Loading in the Moffitt PNAS rna-seq counts (new version)

In [ ]:
pnas_rnaseq = pd.read_csv(r'D:\PNAS2016Data\data_for_release\file_io\PNAS2016_RNAseq_FPKM.csv', sep = ',')
pnas_rnaseq_fin = dict(zip(pnas_rnaseq['geneName'], pnas_rnaseq['FPKM']))
setprna = set(pnas_rnaseq_fin.keys())
genes_to_prna = list(setprna)
pnas_dict_rna = {}
for gene in genes_to_prna:
    pnas_dict_rna[gene] = np.log10(pnas_rnaseq_fin[gene]+1)

threshold of 0 gives all genes, higher threshold closer to the total number of genes gives the lower count genes.

In [ ]:
def getlowcounts(xcounts, ycounts):
    omit = []
    set1 = set(xcounts.keys())
    set2 = set(ycounts.keys())
    genes_to_consider = list(set1.intersection(set2))
    #print(genes_to_consider)
    #genes_to_consider = zcounts
    x = [xcounts[gene] for gene in genes_to_consider if gene not in omit]
    y = [ycounts[gene] for gene in genes_to_consider if gene not in omit]
    z = [math.sqrt(y[z]**2 + x[z]**2) for z in range(len(x))]
    threshold = 0
    temp = []
    while len(temp) < threshold:
        minval = min(z)
        minindex = z.index(minval)
        temp.append(genes_to_consider[minindex])
        del z[minindex]
        del genes_to_consider[minindex]
    return temp

threshold of total gene number gives all genes, lower threshold closer to 0 gives higher count genes.

In [ ]:
def gethighcounts(xcounts, ycounts):
    omit = []
    set1 = set(xcounts.keys())
    set2 = set(ycounts.keys())
    genes_to_consider = list(set1.intersection(set2))
    #genes_to_consider = zcounts
    x = [xcounts[gene] for gene in genes_to_consider if gene not in omit]
    y = [ycounts[gene] for gene in genes_to_consider if gene not in omit]
    z = [math.sqrt(y[z]**2 + x[z]**2) for z in range(len(x))]
    threshold = 130
    temp = []
    while len(temp) < threshold:
        minval = min(z)
        minindex = z.index(minval)
        temp.append(genes_to_consider[minindex])
        del z[minindex]
        del genes_to_consider[minindex]
    t = [gene for gene in genes_to_consider if gene not in temp]
    #t.append('MALAT1')
    return t
        

Loading in our rna-seq values, PNAS MERFISH Counts, and correlating.  Can omit high count or low count genes if wanted.

In [ ]:
#Load RNA-seq rpkm values
col = pd.read_csv(r'D:\MERFISH\Random_Data\U2OS_abundances_qz.rpkm', sep='\t')
#Take the average rpkm of both replicates
col['mean'] = col[['bam.RQZ90.nodup.bam','bam.RQZ91.nodup.bam']].mean(axis=1)
#Log transform the data and turn it into a dictionary
rnaseq_counts = dict(zip(col['Geneid'], np.log10(col['mean']+1)))

new_pnas = pd.read_csv(r'D:\MERFISH\Random_Data\PNAS2016_MERFISH_counts_new.csv', sep=',', header = None)
new_pnas.columns = ['name', 'counts'] 

for i in range(len(pnas['name'])):
    pnas['name'][i] = pnas['name'][i][:-1]

new_pnas_counts = dict(zip(new_pnas['name'], np.log10(new_pnas['counts']+1)))

plot_correlation(xcounts=new_pnas_counts, 
                   ycounts=merfish_counts, 
                   xlabel='Log PNAS MERFISH counts', 
                   ylabel='Log Experiment 4-3D MERFISH Counts', 
                   outfile='RNASeq_corr.png',
                   omit=[])#gethighcounts(xer, yer))

# How a gene behaves in an individual experiment after cellpose and edge cell removing

In [ ]:
#Start here
import glob
import os
import pandas as pd
import numpy as np
import PIL
from PIL import Image


#mask_folder = r'D:\MERFISH\Segmentation\E20190919_R0003\cy3_320diameter_slice30'
#mask_folder = r'D:\MERFISH\Segmentation\E20190920_R0004\cyto_fullsize_320diameter'
#mask_folder = r'D:\MERFISH\Segmentation\E20190930_R0006\cy3_320diameter_slice30'
#mask_folder = r'D:\MERFISH\Segmentation\E20201106_R0034\cyto_fullsize_320diameter'
#mask_folder = r'D:\MERFISH\Segmentation\E20201222_R0049_S1\cyto_fullsize_320diameter'
#mask_folder = r'D:\MERFISH\Segmentation\E20201123_R0040\cyto_fullsize'
mask_folder = r'D:\MERFISH\Segmentation\E20200310_R0022\cy3_320diameter_slice30'

#read in segmentation mask
edgecells = set()
for fov, filename in enumerate(sorted(glob.glob(os.path.join(mask_folder, '*_cp_masks.png')))):
    mask = np.asarray(PIL.Image.open(filename))
    edgecells.update(10000*fov + np.unique(np.concatenate([mask[:,0], mask[:,-1], mask[0,:], mask[-1,:]])))
    
    
#table = pd.read_csv("D:\MERFISH\MERlin_Analysis\E20201106_R0034\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\Archived_Experiments\E20190919_R0003\MERlin_Analysis\E20190919_R0003_v2\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\E20190920_R0004\MERlin_Analysis\E20190920_R0004_v4\single_cell_analysis_cyto_fullsize_320diameter\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\Archived_Experiments\E20190930_R0006\MERlin_Analysis\E20190930_R0006\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\MERlin_Analysis\E20201106_R0035\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\MERlin_Analysis\E20201223_R0049_S2\single_cell_analysis_cyto_fullsize_320diameter\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\MERlin_Analysis\E20201222_R0049_S1\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\MERlin_Analysis\E20201123_R0040\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\E20200310_R0022\MERlin_Analysis\E20200310_R0022\single_cell_analysis_cy3_320diameter_slice30\single_cell_raw_counts.csv")


table = pd.read_csv("D:\MERFISH\E20200310_R0022\MERlin_Analysis\E20200310_R0022\single_cell_analysis_cy3_320diameter_slice30\single_cell_raw_counts.csv")
#filter out the cells on the edges
table = table[~table['cell_id'].isin(edgecells)] 
for index, row in table.iterrows():
    total = sum(row[1:])
    for i in range(len(row)):
        if i != 0:
            row[i] = float(row[i]/total)*1000000
        

In [ ]:
import statistics
import numpy as np
import pandas as pd
import matplotlib
#Get the specific gene from the table and perform calculations
prp_vals = table["NOTCH2"].to_list()
average_per_cell = sum(prp_vals)/len(prp_vals)
num_cells = len(prp_vals)
std_dev_cell = statistics.stdev(prp_vals)

#The total number of gene counts across all cells
print(sum(prp_vals))
#The total number of cells after filtering in the experiment
print(num_cells)
#The average number of gene counts per cell
print(average_per_cell)
#The standard deviation of the number of gene counts per cell
print(std_dev_cell)
#The standard deviation of the number of gene counts per cell divided by the average number of gene counts per cell
print(std_dev_cell/average_per_cell)

histo = pd.DataFrame({'Experiment 22 NOTCH2 Counts Per Cell': table["NOTCH2"].to_list()})
plot = histo.plot.hist(bins = 100)
fig = plot.get_figure()

Assorted gene metrics

In [ ]:
genetypes = table.columns[1:]
prpvals_list = []
average_per_cell_list = []
num_cells_list = []
std_list = []
for i in genetypes:
    prp_vals = table[i].to_list()
    average_per_cell = sum(prp_vals)/len(prp_vals)
    num_cells = len(prp_vals)
    std_dev_cell = statistics.stdev(prp_vals)
    prpvals_list.append(sum(prp_vals))
    average_per_cell_list.append(average_per_cell)
    num_cells_list.append(num_cells)
    std_list.append(std_dev_cell)
    

In [ ]:
plot = pd.DataFrame({'Experiment 22 All genes total number of gene counts': prpvals_list}).plot.hist(bins = 100)
fig = plot.get_figure()


In [ ]:
plot = pd.DataFrame({'Experiment 22 average number of gene counts per cell after filtering': average_per_cell_list}).plot.hist(bins = 100)
fig = plot.get_figure()

In [ ]:
plot = pd.DataFrame({'Experiment 22 standard deviation of gene counts per cell after filtering': std_list}).plot.hist(bins = 100)
fig = plot.get_figure()

In [ ]:
plot = pd.DataFrame({'Experiment 22 standard deviation of gene counts per cell after filtering': std_list}).plot.hist(bins = 100)
fig = plot.get_figure()

# Viewing all experiments at once

In [ ]:

mask_folders = ['D:\MERFISH\Segmentation\E20190919_R0003\cy3_320diameter_slice30','D:\MERFISH\Segmentation\E20190920_R0004\cyto_fullsize_320diameter', 'D:\MERFISH\Segmentation\E20190930_R0006\cy3_320diameter_slice30', 'D:\MERFISH\Segmentation\E20201106_R0034\cyto_fullsize_320diameter', 'D:\MERFISH\Segmentation\E20201222_R0049_S1\cyto_fullsize_320diameter', 'D:\MERFISH\Segmentation\E20201123_R0040\cyto_fullsize', 'D:\MERFISH\Segmentation\E20200310_R0022\cy3_320diameter_slice30', 'D:\MERFISH\Segmentation\E20201223_R0049_S2\cyto_fullsize_320diameter']
raw_counts_folders = ["D:\MERFISH\Archived_Experiments\E20190919_R0003\MERlin_Analysis\E20190919_R0003_v2\single_cell_raw_counts.csv", "D:\MERFISH\E20190920_R0004\MERlin_Analysis\E20190920_R0004_v4\single_cell_analysis_cyto_fullsize_320diameter\single_cell_raw_counts.csv", "D:\MERFISH\Archived_Experiments\E20190930_R0006\MERlin_Analysis\E20190930_R0006\single_cell_raw_counts.csv", "D:\MERFISH\MERlin_Analysis\E20201106_R0034\single_cell_raw_counts.csv", "D:\MERFISH\MERlin_Analysis\E20201222_R0049_S1\single_cell_raw_counts.csv", "D:\MERFISH\MERlin_Analysis\E20201123_R0040\single_cell_raw_counts.csv", "D:\MERFISH\E20200310_R0022\MERlin_Analysis\E20200310_R0022\single_cell_analysis_cy3_320diameter_slice30\single_cell_raw_counts.csv", "D:\MERFISH\MERlin_Analysis\E20201223_R0049_S2\single_cell_analysis_cyto_fullsize_320diameter\single_cell_raw_counts.csv"]

In [ ]:
highlyexpressed = ['SPTBN1', 'COL5A1', 'HERC2', 'FASN', 'ANKRD52', 'MALAT1', 'PRKDC', 'FAM208B', 'USP9X', 'UBR5', 'TPR', 'LRP1', 'PLXNA1', 'CHST3', 'FAF2', 'PRPF8', 'CENPF', 'TLN1', 'AFF4', 'THBS1', 'DYNC1H1', 'ZNF592', 'CREBBP', 'PRRC2B', 'USP34', 'CKAP5', 'SRRM2', 'CBX5', 'FBN2', 'NOTCH2']
lowlyexpressed = ['TMOD2', 'USP8', 'SMIM5', 'FAM184B', 'RBM20', 'CEMIP', 'BSN', 'MCF2L', 'SLC9A2', 'XKR5', 'SAMD12', 'SULT1C2', 'TSPAN3', 'NHSL2', 'THSD4', 'RP4-671O14.6', 'SCUBE3', 'PAPPA', 'C14orf132', 'AGO3', 'SOD2', 'SKP1', 'RNF152', 'KIAA1462', 'XDH', 'SLC35B4', 'DOPEY1', 'DSEL', 'CNR2', 'RAD51D']
counts_sum_per_gene_all_cells = []
gene_average_counts_per_cell = []
std_deviation_per_gene = []
divided_per_gene = []
num_cells_exp = []
order = ['Exp 3', 'Exp 4', 'Exp 6', 'Exp 34', 'Exp 49_S1', 'Exp 40', 'Exp 22', 'Exp 49_S2']
for i in range(len(mask_folders)):
    mask_folder = mask_folders[i]
    table = pd.read_csv(raw_counts_folders[i])
    
    edgecells = set()
    for fov, filename in enumerate(sorted(glob.glob(os.path.join(mask_folder, '*_cp_masks.png')))):
        mask = np.asarray(PIL.Image.open(filename))
        edgecells.update(10000*fov + np.unique(np.concatenate([mask[:,0], mask[:,-1], mask[0,:], mask[-1,:]])))
    
    table.head()
    table = table[~table['cell_id'].isin(edgecells)] 
    for index, row in table.iterrows():
        total = sum(row[1:])
        for i in range(len(row)):
            if i != 0:
                row[i] = float(row[i]/total)*1000000
        
    prpvals_list = []
    average_per_cell_list = []
    num_cells_list = []
    std_list = []
    divided_list = []
    for i in highlyexpressed:
    #for i in genetypes:
        prp_vals = table[i].to_list()
        average_per_cell = sum(prp_vals)/len(prp_vals)
        num_cells = len(prp_vals)
        std_dev_cell = statistics.stdev(prp_vals)
        prpvals_list.append(sum(prp_vals))
        average_per_cell_list.append(average_per_cell)
        num_cells_list.append(num_cells)
        std_list.append(std_dev_cell)
        divided_list.append(std_dev_cell/average_per_cell)
    counts_sum_per_gene_all_cells.append(np.mean(prpvals_list))
    gene_average_counts_per_cell.append(np.mean(average_per_cell_list))
    std_deviation_per_gene.append(np.mean(std_list))
    divided_per_gene.append(np.mean(divided_list))
    num_cells_exp.append(np.mean(num_cells))

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(order,counts_sum_per_gene_all_cells)
plt.title('Average sum of counts per gene across all cells')
plt.show()

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(order,gene_average_counts_per_cell)
plt.title('Average counts per gene per cell')
plt.show()

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(order,std_deviation_per_gene)
plt.title('Average standard deviation of gene counts across cells per experiment')
plt.show()

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(order,divided_per_gene)
plt.title('Average (standard deviation/average gene counts) across cells per experiment')
plt.show()

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(order,num_cells_exp)
plt.title('Average number of cells per experiment')
plt.show()

# Running Cellpose Correlations

In [ ]:
highlyexpressed = ['SPTBN1', 'COL5A1', 'HERC2', 'FASN', 'ANKRD52', 'MALAT1', 'PRKDC', 'FAM208B', 'USP9X', 'UBR5', 'TPR', 'LRP1', 'PLXNA1', 'CHST3', 'FAF2', 'PRPF8', 'CENPF', 'TLN1', 'AFF4', 'THBS1', 'DYNC1H1', 'ZNF592', 'CREBBP', 'PRRC2B', 'USP34', 'CKAP5', 'SRRM2', 'CBX5', 'FBN2', 'NOTCH2']
lowlyexpressed = ['TMOD2', 'USP8', 'SMIM5', 'FAM184B', 'RBM20', 'CEMIP', 'BSN', 'MCF2L', 'SLC9A2', 'XKR5', 'SAMD12', 'SULT1C2', 'TSPAN3', 'NHSL2', 'THSD4', 'RP4-671O14.6', 'SCUBE3', 'PAPPA', 'C14orf132', 'AGO3', 'SOD2', 'SKP1', 'RNF152', 'KIAA1462', 'XDH', 'SLC35B4', 'DOPEY1', 'DSEL', 'CNR2', 'RAD51D']
counts_sum_per_gene_all_cells = []
gene_average_counts_per_cell = []
std_deviation_per_gene = []
divided_per_gene = []
num_cells_exp = []
order = ['Exp 3', 'Exp 4', 'Exp 6', 'Exp 34', 'Exp 49_S1', 'Exp 40', 'Exp 22', 'Exp 49_S2']
for i in range(len(mask_folders)):
    mask_folder = mask_folders[i]
    table = pd.read_csv(raw_counts_folders[i])
    
    edgecells = set()
    for fov, filename in enumerate(sorted(glob.glob(os.path.join(mask_folder, '*_cp_masks.png')))):
        mask = np.asarray(PIL.Image.open(filename))
        edgecells.update(10000*fov + np.unique(np.concatenate([mask[:,0], mask[:,-1], mask[0,:], mask[-1,:]])))
    
    table.head()
    table = table[~table['cell_id'].isin(edgecells)] 
    for index, row in table.iterrows():
        total = sum(row[1:])
        for i in range(len(row)):
            if i != 0:
                row[i] = float(row[i]/total)*1000000
        
    prpvals_list = []
    average_per_cell_list = []
    num_cells_list = []
    std_list = []
    divided_list = []
    for i in genetypes:
    #for i in genetypes:
        prp_vals = table[i].to_list()
        average_per_cell = sum(prp_vals)/len(prp_vals)
        num_cells = len(prp_vals)
        std_dev_cell = statistics.stdev(prp_vals)
        prpvals_list.append(sum(prp_vals))
        average_per_cell_list.append(average_per_cell)
        num_cells_list.append(num_cells)
        std_list.append(std_dev_cell)
        divided_list.append(std_dev_cell/average_per_cell)
    counts_sum_per_gene_all_cells.append(prpvals_list)
    gene_average_counts_per_cell.append(average_per_cell_list)
    std_deviation_per_gene.append(std_list)
    divided_per_gene.append(divided_list)
    num_cells_exp.append(num_cells)

In [ ]:
keyz = ['Blank-1','Blank-10','Blank-2','Blank-3','Blank-4','Blank-5','Blank-6','Blank-7','Blank-8','Blank-9']
for i in keyz:
    del new_pnas_counts[i]

In [ ]:
mercounts = {}
for v in range(len(counts_sum_per_gene_all_cells)):
    cts = np.log10(counts_sum_per_gene_all_cells[v])
    for i in range(len(cts)):
        mercounts[genetypes[i]] = float(cts[i])

    plot_correlation(xcounts=new_pnas_counts,#pnas_dict_rna,#new_pnas_counts, 
                       ycounts=mercounts, #zcounts=genes_to_consider7,
                       xlabel='Log PNAS MERFISH counts', 
                       ylabel='Log Experiment ' + order[v] + ' MERFISH Counts', 
                       outfile='RNASeq_corr.png',
                       omit=[])

# Relative Expression Rank Comparison

In [ ]:
topvals = {}
for i in range(len(mask_folders)):
    mask_folder = mask_folders[i]
    table = pd.read_csv(raw_counts_folders[i])
    
    edgecells = set()
    for fov, filename in enumerate(sorted(glob.glob(os.path.join(mask_folder, '*_cp_masks.png')))):
        mask = np.asarray(PIL.Image.open(filename))
        edgecells.update(10000*fov + np.unique(np.concatenate([mask[:,0], mask[:,-1], mask[0,:], mask[-1,:]])))
    
    table.head()
    table = table[~table['cell_id'].isin(edgecells)] 
    for index, row in table.iterrows():
        total = sum(row[1:])
        for i in range(len(row)):
            if i != 0:
                row[i] = float(row[i]/total)*1000000
    
    ziplist_gene = []
    for k in genetypes:
        pp = table[k].tolist()
        pp = sum(pp)/len(pp)
        ziplist_gene.append(pp)
        
    ziplist_percent = []

    for z in ziplist_gene:
        countt = 0
        for b in ziplist_gene:
            if b > z:
                countt += 1
        ziplist_percent.append(countt/len(ziplist_gene))       
    
    for k in range(len(genetypes)):  
        if genetypes[k] in topvals:   
            topvals[genetypes[k]].append(ziplist_percent[k])
        else:
            topvals[genetypes[k]] = [ziplist_percent[k]]
    

In [ ]:
for gene in topvals:    
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.bar(order,topvals.get(gene))
    plt.title('Percent counts higher than ' + gene)
    plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(order,topvals.get('SOD2'))
plt.title('Percent counts higher than SOD2')# + gene)
plt.show()

t-test gel-embedded non-embedded expression rank

In [ ]:
import scipy.stats as sst

df = pd.DataFrame([statistics.stdev(topvals.get(n)) for n in topvals])
df.columns = ['Std Dev']
df.index = [n for n in topvals]
df.sort_values(by = ['Std Dev'], axis = 0)

df13 = pd.DataFrame([statistics.stdev(topvals.get(n)[:3]) for n in topvals])
df13.columns = ['Std Dev']
df13.index = [n for n in topvals]
df13=np.sort(df13['Std Dev'])


ttest = sst.ttest_ind(df, df13)
print(ttest)

# RNAseq Relative Expression Rank

In [ ]:

pnas_dict_rna_names = [i for i in pnas_dict_rna]
pnas_dict_rna_values = [pnas_dict_rna.get(i) for i in pnas_dict_rna]
df_rna = pd.DataFrame(zip(pnas_dict_rna_names, pnas_dict_rna_values))
df_rna.columns = ['gene', 'ct']
df_rna = df_rna.sort_values(by=['ct'])
df_rna['higher_than'] = [i for i in range(len(df_rna['gene']))]

df_rna_dict = {}
for i in range(len(df_rna['gene'])):
    df_rna_dict[df_rna['gene'][i]] = df_rna['higher_than'][i]
    
vals = [topvals.get(n)[0] for n in topvals]
genes = [n for n in topvals]
df_rna2 = pd.DataFrame(zip(genes, vals))
df_rna2.columns = ['gene', 'ct']
df_rna2 = df_rna2.sort_values(by=['ct'])
df_rna2['higher_than'] = [(i-129)*-1 for i in range(len(df_rna2['gene']))]

df_mer_dict = {}
for i in range(len(df_rna2['gene'])):
    df_mer_dict[df_rna2['gene'][i]] = df_rna2['higher_than'][i]

In [ ]:
def plot_correlation2(xcounts, ycounts, xlabel, ylabel, outfile, omit=[]):
    set1 = set(xcounts.keys())
    set2 = set(ycounts.keys())
    genes_to_consider = list(set1.intersection(set2))
    #genes_to_consider = zcounts
    x = [xcounts[gene] for gene in genes_to_consider if gene not in omit]
    y = [ycounts[gene] for gene in genes_to_consider if gene not in omit]
    using = []
    #using.append("PRPF8")
    #using.append("BRCA2")
    #using.append("MALAT1")
    #using.append("EGFR")
    #using.append("CENPF")
    #using.append("MED14")
    #using.append("CBX5")
    #using.append("SMARCA5")
    #using.append("IGF2R")
    #using.append("FBN2")
    #using.append("THBS1")
    for gene in genes_to_consider: 
        if gene not in omit and gene in using:
            print(gene, (10**ycounts[gene]))
    num = []
    count = 0
    for gene in genes_to_consider:
    #    if gene not in omit:
    #        count += 1
            #print(gene)
    #        if gene == "EGFR":
    #            print(10**y[count])
            if gene not in omit:
                num.append(gene) 
    
    corr, pval = pearsonr(x,y)
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.figure(figsize=(7,5))
    plt.plot(x,p(x),"r--")

    plt.scatter(x, y)
    plt.title("Pearson = %.3f" %corr)
    plt.ylabel(ylabel)
    plt.xlim(-5, 135)
    plt.ylim(-5, 135)
    plt.xlabel(xlabel)
    for i, txt in enumerate(num):
        plt.annotate(txt, (x[i], y[i]), size=8)#, textcoords= "figure fraction")
    #plt.savefig(os.path.join(analysis_folder, outfile), dpi=300)
    

In [ ]:
plot_correlation2(xcounts=df_rna_dict,#pnas_dict_rna,#new_pnas_counts, 
                    ycounts=df_mer_dict, #zcounts=genes_to_consider7,
                    xlabel='PNAS RNA Rank', 
                    ylabel='MERFISH Counts Rank Exp 4', 
                    outfile='RNASeq_corr.png',
                    omit=[])

# Find 2 bit similarities

Enter gene name below

In [ ]:
gene2bit = "SOD2"
codebook = pandas.read_csv(codebook_file)
codebook2 = codebook[~codebook['name'].str.contains("blank")]
codebook2 = codebook2[~codebook2['name'].str.contains("notarget")]
codebook_dict = {}
for index, codebook2r in codebook2.iterrows():

    addlist = [codebook2r.bit1, codebook2r.bit2, codebook2r.bit3, codebook2r.bit4, codebook2r.bit5, codebook2r.bit6, codebook2r.bit7, codebook2r.bit8, codebook2r.bit9, codebook2r.bit10, codebook2r.bit11, codebook2r.bit12, codebook2r.bit13, codebook2r.bit14, codebook2r.bit15, codebook2r.bit16]
    codebook_dict[codebook2r.id] = addlist

spots = [i+1 for i in range(len(codebook_dict.get(gene2bit))) if codebook_dict.get(gene2bit)[i] == 1]
spots

In [ ]:
code_id = codebook2['id'].tolist()
genes_the_same = []
bits_the_same = []
for ij in code_id:
    if ij == gene2bit:
        continue
    else:
        spots_compare = [i for i in range(len(codebook_dict.get(ij))) if codebook_dict.get(ij)[i] == 1]
        spots_compare = [k+1 for k in spots_compare if k+1 in spots]

        if len(spots_compare) >= 2:
            genes_the_same.append(ij)
            bits_the_same.append([e for e in spots_compare])
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(order,topvals.get(gene2bit))
plt.title('Percent counts higher than ' + gene2bit)# + gene)
plt.show()

Viewing specific similar bits as outputted 2 cells above

In [ ]:
bcount = 0
#enter the gene bits you want to look at i.e [1, 7]
bits_included = [16]
for f in genes_the_same:
    if len([i for i in bits_included if i in bits_the_same[bcount]]) == len(bits_included):
        fig = plt.figure()
        ax = fig.add_axes([0,0,1,1])
        ax.bar(order,topvals.get(f))
        plt.title('Percent counts higher than ' + f + ', Bits: ' + str(bits_the_same[bcount]))# + gene)
        plt.show()
    bcount += 1

# Find total experiment statistics by looking at the raw counts for all genes at once per cell

In [ ]:
import glob
import os
import pandas as pd
import numpy as np
from PIL import Image
#mask_folder = r'D:\MERFISH\Segmentation\E20190919_R0003\cy3_320diameter_slice30'
#mask_folder = r'D:\MERFISH\Segmentation\E20190920_R0004\cyto_fullsize_320diameter'
#mask_folder = r'D:\MERFISH\Segmentation\E20190930_R0006\cy3_320diameter_slice30'
#mask_folder = r'D:\MERFISH\Segmentation\E20201106_R0034\cyto_fullsize_320diameter'
#mask_folder = r'D:\MERFISH\Segmentation\E20201222_R0049_S1\cyto_fullsize_320diameter'
#mask_folder = r'D:\MERFISH\Segmentation\E20201123_R0040\cyto_fullsize'
mask_folder = r'D:\MERFISH\Segmentation\E20201106_R0035\cyto_fullsize_320diameter'

edgecells = set()
for fov, filename in enumerate(sorted(glob.glob(os.path.join(mask_folder, '*_cp_masks.png')))):
    mask = np.asarray(PIL.Image.open(filename))
    edgecells.update(10000*fov + np.unique(np.concatenate([mask[:,0], mask[:,-1], mask[0,:], mask[-1,:]])))
#table = pd.read_csv("D:\MERFISH\MERlin_Analysis\E20201106_R0034\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\Archived_Experiments\E20190919_R0003\MERlin_Analysis\E20190919_R0003_v2\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\E20190920_R0004\MERlin_Analysis\E20190920_R0004_v4\single_cell_analysis_cyto_fullsize_320diameter\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\Archived_Experiments\E20190930_R0006\MERlin_Analysis\E20190930_R0006\single_cell_raw_counts.csv")
table = pd.read_csv("D:\MERFISH\MERlin_Analysis\E20201106_R0035\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\MERlin_Analysis\E20201223_R0049_S2\single_cell_analysis_cyto_fullsize_320diameter\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\MERlin_Analysis\E20201222_R0049_S1\single_cell_raw_counts.csv")
#table = pd.read_csv("D:\MERFISH\MERlin_Analysis\E20201123_R0040\single_cell_raw_counts.csv")
table = table[~table['cell_id'].isin(edgecells)] 
#for index, row in table.iterrows():
#    total = sum(row[1:])
#    for i in range(len(row)):
#        if i != 0:
#            row[i] = float(row[i]/total)*1000000
import statistics 

table["TotalVals"] = table[list(table.columns)].sum(axis=1)
total_vals = table["TotalVals"].to_list()
average_per_cell = sum(total_vals)/len(total_vals)
num_cells = len(total_vals)
std_dev_cell = statistics.stdev(total_vals)

print(sum(total_vals))
#print(sum(prp_vals))
print(num_cells)
print(average_per_cell)
print(std_dev_cell)
print(std_dev_cell/average_per_cell)
import numpy as np
import pandas as pd
import matplotlib
histo = pd.DataFrame({'Experiment 40 Counts Per Cell': total_vals})
binwidth = max(total_vals)/40
plot = histo.plot.hist( bins=np.arange(min(total_vals), max(total_vals) + binwidth, binwidth))
fig = plot.get_figure()        